# EXAMPLE GET INLIERS
In this notebook, we ...
As ouput, the method generates a selection of nodes 

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [2]:
#IMPORT PACKAGES
from rdflib import Graph

import os.path, time
import importlib
import numpy as np
import open3d as o3d
import ifcopenshell
from ifcopenshell.util.selector import Selector

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

## 1. INPUT

In [5]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
sessionPath = os.path.join(projectPath,"Samples11") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon
classes= '.IfcBeam | .IfcColumn | .IfcWall | .IfcSlab'

t_d =10.0 # (float) search distance
t_x=2.0 # (float) search distance in X given the boundingbox
t_y=2.0 # (float) search distance in Y given the boundingbox
t_z=1.0 # (float) search distance in Z given the boundingbox

idex=2 # index of the node you want to test

In [7]:
meshPath=os.path.join(sessionPath,"week22.obj")  
mesh= o3d.io.read_triangle_mesh(meshPath)

In [8]:
pcdPath=os.path.join(sessionPath,"academiestraat week 22 19.pcd")  
pcd= o3d.io.read_point_cloud(pcdPath) 

## 2. READ DATA

In [9]:
ifcPath2=os.path.join(sessionPath,"ifc_parking.ifc") 
classes= '.IfcBeam | .IfcColumn | .IfcWall | .IfcSlab'
ifc2 = ifcopenshell.open(ifcPath2)   

ifcSlab=ifc2.by_guid('2qZtnImXH6Tgdb58DjNlmF')
ifcWall=ifc2.by_guid('06v1k9ENv8DhGMCvKUuLQV')
ifcBeam=ifc2.by_guid('05Is7PfoXBjhBcbRTnzewz' )
ifcColumn=ifc2.by_guid('23JN72MijBOfF91SkLzf3a')
# ifcWindow=ifc.by_guid(cls.slabGlobalid) 
# ifcDoor=ifc.by_guid(cls.slabGlobalid)

slabMesh=gmu.ifc_to_mesh(ifcSlab)
wallMesh=gmu.ifc_to_mesh(ifcWall)
beamMesh=gmu.ifc_to_mesh(ifcBeam)
columnMesh=gmu.ifc_to_mesh(ifcColumn)

In [10]:
ifcPath1=os.path.join(sessionPath,"ifc.ifc") 
ifc1 = ifcopenshell.open(ifcPath1)   
selector = Selector() 
bimMeshes=[gmu.ifc_to_mesh(ifcElement) for ifcElement in selector.parse(ifc1, classes)]     
bimBoxes=[mesh.get_oriented_bounding_box() for mesh in bimMeshes if mesh]

In [11]:
for box in bimBoxes:
    box.color=[0,1,0]

In [12]:
o3d.visualization.draw_geometries(bimBoxes+[mesh]+bimMeshes)

## test_get_box_inliers

In [13]:
wallMesh.translate([0,0,3])
wallBox=wallMesh.get_oriented_bounding_box()
wallInliers= gmu.get_box_inliers(sourceBox=wallBox, testBoxes=bimBoxes)
print(len(wallInliers))

8


In [14]:
selectedBoxes=[ bimBoxes[i] for i in wallInliers]
print(len(selectedBoxes))
for box in selectedBoxes:
    box.color=[1,0,0]
o3d.visualization.draw_geometries(bimBoxes +selectedBoxes+ [wallBox] )

8


## test_get_box_intersections

In [15]:
mesh=wallMesh.translate([0,0,3])
wallBox=mesh.get_oriented_bounding_box()
wallInliers= gmu.get_box_intersections(sourceBox=wallBox, testBoxes=bimBoxes)
print(len(wallInliers))

41


In [16]:
selectedBoxes=[ bimBoxes[i] for i in wallInliers]
for box in selectedBoxes:
    box.color=[1,0,0]
o3d.visualization.draw_geometries(bimBoxes +selectedBoxes+ [wallBox] )

## test_get_mesh_collisions_trimesh

In [17]:
meshes=bimMeshes
inliers=gmu.get_mesh_collisions_trimesh(sourceMesh=mesh , geometries =meshes) 
print(len(inliers))


30


In [18]:
selected=[ meshes[i] for i in inliers]
for s in selected:
    s.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries( [mesh] + selected )

## test_get_pcd_collisions

In [19]:
meshes=[mesh.sample_points_uniformly(1000) for mesh in bimMeshes]
inliers=gmu.get_pcd_collisions(sourcePcd=pcd , geometries =meshes) 
print(len(inliers))

124


In [20]:
sourceHull, _ = pcd.compute_convex_hull()
sourceHull.paint_uniform_color([1,0,0])

TriangleMesh with 85 points and 166 triangles.

In [21]:
selected=[ meshes[i] for i in inliers]
for s in selected:
    s.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries( [pcd] + selected +[sourceHull])

## test_get_rotation_matrix

In [22]:
from scipy.spatial.transform import Rotation as R

box=mesh.get_oriented_bounding_box()
orientedBounds=np.asarray(box.get_box_points())
print(orientedBounds)
# rotationMatrix=np.asarray(box.R)
rotationMatrix=np.array([[-0.0544245051021791, 0.089782180920334, 0.994473294379276],
                                                [-0.783686718502243, -0.621016494566922, 0.0131772804097903],
                                                [0.618767404189627, -0.778638345745315, 0.104159618122526]])


[[-19.83941221  90.32921075  12.45      ]
 [-12.98842887  64.40506348  12.45      ]
 [-19.83941221  90.32921075   5.75      ]
 [-19.25652179  90.48325141  12.45      ]
 [-12.40553844  64.55910413   5.75      ]
 [-19.25652179  90.48325141   5.75      ]
 [-12.40553844  64.55910413  12.45      ]
 [-12.98842887  64.40506348   5.75      ]]
